In [19]:
import duckdb

# create a connection to a file called 'test_database.db'
con = duckdb.connect("../test_database.db")

In [22]:
con.sql("SELECT * FROM crosswalk_zillow")

┌────────────┬────────────────┬───────────┬────────────┬──────────────────┬───────────────────────────────────────┬───────────────────────┬──────────────────────┬───────┬──────────┐
│ CountyName │   StateName    │ StateFIPS │ CountyFIPS │ MetroName_Zillow │               CBSAName                │ CountyRegionID_Zillow │ MetroRegionID_Zillow │ FIPS  │ CBSACode │
│  varchar   │    varchar     │   int64   │   int64    │     varchar      │                varchar                │         int64         │        double        │ int64 │  double  │
├────────────┼────────────────┼───────────┼────────────┼──────────────────┼───────────────────────────────────────┼───────────────────────┼──────────────────────┼───────┼──────────┤
│ Pike       │ Pennsylvania   │        42 │        103 │ New York, NY     │ New York-Newark-Jersey City, NY-NJ-PA │                   280 │             394913.0 │ 42103 │  35620.0 │
│ Bronx      │ New York       │        36 │          5 │ New York, NY     │ New York-Newar

In [23]:
con.sql("""
CREATE OR REPLACE TABLE crosswalk_zillow_tract AS 
WITH 
    disinct_ziptract AS (SELECT ZIP AS zip, TRACT AS tract FROM crosswalk_zip_tract GROUP BY ZIP, TRACT),
    disinct_cbsazip AS (SELECT CBSA AS cbsa, ZIP AS zip, FROM crosswalk_cbsa_zip GROUP BY CBSA, ZIP),
    disinct_zillowcbsa AS (SELECT MetroRegionID_Zillow AS zillow_code, MetroName_Zillow as zillow_name, CBSACode AS cbsa FROM crosswalk_zillow GROUP BY MetroRegionID_Zillow, MetroName_Zillow, CBSACode),
    zillow_zip AS (SELECT zillow_code, disinct_zillowcbsa.cbsa, disinct_zillowcbsa.zillow_name, disinct_cbsazip.zip FROM disinct_zillowcbsa LEFT JOIN disinct_cbsazip ON disinct_zillowcbsa.cbsa = disinct_cbsazip.cbsa)
SELECT zillow_code, zillow_name, tract FROM zillow_zip LEFT JOIN disinct_ziptract ON zillow_zip.zip = disinct_ziptract.zip GROUP BY zillow_code, zillow_name, tract
""")

In [24]:
con.sql("show tables")

┌────────────────────────┐
│          name          │
│        varchar         │
├────────────────────────┤
│ crosswalk_cbsa_zip     │
│ crosswalk_zillow       │
│ crosswalk_zillow_tract │
│ crosswalk_zip_tract    │
│ svi                    │
│ zillow_mean_to_pending │
└────────────────────────┘

In [25]:
con.sql("SELECT * FROM crosswalk_zillow_tract LIMIT 10")

┌─────────────┬───────────────────────────┬─────────────┐
│ zillow_code │        zillow_name        │    tract    │
│   double    │          varchar          │    int64    │
├─────────────┼───────────────────────────┼─────────────┤
│    394404.0 │ Boston, MA                │ 25017339900 │
│    394669.0 │ Hartford, CT              │  9003473501 │
│    394887.0 │ Mount Sterling, KY        │ 21173920400 │
│    394856.0 │ Miami-Fort Lauderdale, FL │ 12099003507 │
│    394459.0 │ Charlottesville, VA       │ 51540000700 │
│    753924.0 │ Urban Honolulu, HI        │ 15003005900 │
│    395209.0 │ Washington, DC            │ 51013102302 │
│    395209.0 │ Washington, DC            │ 51013102400 │
│    394306.0 │ Albany, GA                │ 13095000900 │
│    394306.0 │ Albany, GA                │ 13095011300 │
├─────────────┴───────────────────────────┴─────────────┤
│ 10 rows                                     3 columns │
└───────────────────────────────────────────────────────┘

In [26]:
con.commit()
con.close()